In [51]:
# Import necessary libraries
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim

# Define transform to normalize data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [52]:
# Load the dataset
train_data = datasets.ImageFolder(root='./data', transform=transform)

# Create data loader
trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)


In [53]:
import torch
import torch.nn as nn

class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        # Define your layers here
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 256)  # Assuming input image size is 224x224
        self.fc2 = nn.Linear(256, 10)  # Adjust num_classes according to your dataset

    def forward(self, x):
        # Define the forward pass
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
model = YourModel()



In [54]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [55]:
# Number of epochs
num_epochs = 50



In [56]:
# Train the model
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


Finished Training


In [66]:
# Evaluate the model on the training dataset
correct_train = 0
total_train = 0
with torch.no_grad():
    for data in trainloader:  # Assuming trainloader is defined for training dataset
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

train_accuracy = 100 * correct_train / total_train
print('Accuracy of the network on the training images: %d %%' % train_accuracy)


Accuracy of the network on the training images: 100 %


In [57]:
# Save the trained model
torch.save(model.state_dict(), 'your_model.pth')


In [58]:
# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize your model
model = YourModel().to(device)

# Print device
print("Model is running on:", device)


Model is running on: cpu


In [59]:
# Print the architecture of your model
print(model)

# Compare with the keys in the loaded state dictionary
print("Keys in the loaded state dictionary:", state_dict.keys())


YourModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
Keys in the loaded state dictionary: odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


In [60]:
# Load the model architecture
model = YourModel()

# Load the state dictionary
state_dict = torch.load("your_model.pth", map_location=torch.device('cpu'))

# Print the architecture of your model
print(model)

# Compare with the keys in the loaded state dictionary
print("Keys in the loaded state dictionary:", state_dict.keys())


YourModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
Keys in the loaded state dictionary: odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


In [61]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load the image
# Load the image
# Load the image
image_path = "hrx.jpeg"  # Replace with the actual path to your test image
test_image = Image.open(image_path)



# Define transform to preprocess the test image
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Preprocess the test image
processed_image = test_transform(test_image).unsqueeze(0)  # Add a batch dimension

# Load your trained model
model = YourModel().to(device)  # Assuming device is defined as in previous code snippets

# Load the model weights
model.load_state_dict(torch.load("your_model.pth"))  # Replace with the path to your saved model

# Set the model to evaluation mode
model.eval()

# Pass the processed image through the model
with torch.no_grad():
    output = model(processed_image.to(device))

# Interpret the model's output
probabilities = torch.softmax(output, dim=1)
predicted_class = torch.argmax(probabilities).item()

# Print the predicted class
print("Predicted class:", predicted_class)


Predicted class: 5


In [65]:
# Load a single test image and label
image_path = "hrx.jpeg"  # Replace with the path to your test image
test_image = Image.open(image_path)
test_image = transform(test_image).unsqueeze(0)  # Add batch dimension and apply transformation

# Load the corresponding label (assuming it's known)
# Replace `true_label` with the actual label of the test image
true_label = 0  # Replace 0 with the actual label

# Evaluate the model on the single test image
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    output = model(test_image)
    _, predicted = torch.max(output, 1)
    if predicted.item() == true_label:
        print("Correct prediction!")
    else:
        print("Incorrect prediction.")

# Calculate accuracy
if predicted.item() == true_label:
    accuracy = 100
else:
    accuracy = 0
print('Accuracy of the network on the single test image: %d %%' % accuracy)


Incorrect prediction.
Accuracy of the network on the single test image: 0 %
